In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re # for regex
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score
import pickle

In [3]:
data = pd.read_csv('/IMDB-Dataset.csv')
print(data.shape)
data.head()

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [5]:
data.sentiment.value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [6]:
data.sentiment.replace('positive',1,inplace=True)
data.sentiment.replace('negative',0,inplace=True)
data.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
5,"Probably my all-time favorite movie, a story o...",1
6,I sure would like to see a resurrection of a u...,1
7,"This show was an amazing, fresh & innovative i...",0
8,Encouraged by the positive comments about this...,0
9,If you like original gut wrenching laughter yo...,1


In [7]:
data.review[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [9]:
def clean(text):
    cleaned = re.compile(r'<.*?>')
    return re.sub(cleaned,'',text)

data.review = data.review.apply(clean)
data.review[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows wo

In [10]:
def is_special(text):
    rem = ''
    for i in text:
        if i.isalnum():
            rem = rem + i
        else:
            rem = rem + ' '
    return rem

data.review = data.review.apply(is_special)
data.review[0]

'One of the other reviewers has mentioned that after watching just 1 Oz episode you ll be hooked  They are right  as this is exactly what happened with me The first thing that struck me about Oz was its brutality and unflinching scenes of violence  which set in right from the word GO  Trust me  this is not a show for the faint hearted or timid  This show pulls no punches with regards to drugs  sex or violence  Its is hardcore  in the classic use of the word It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary  It focuses mainly on Emerald City  an experimental section of the prison where all the cells have glass fronts and face inwards  so privacy is not high on the agenda  Em City is home to many  Aryans  Muslims  gangstas  Latinos  Christians  Italians  Irish and more    so scuffles  death stares  dodgy dealings and shady agreements are never far away I would say the main appeal of the show is due to the fact that it goes where other shows wo

In [11]:
def to_lower(text):
    return text.lower()

data.review = data.review.apply(to_lower)
data.review[0]

'one of the other reviewers has mentioned that after watching just 1 oz episode you ll be hooked  they are right  as this is exactly what happened with me the first thing that struck me about oz was its brutality and unflinching scenes of violence  which set in right from the word go  trust me  this is not a show for the faint hearted or timid  this show pulls no punches with regards to drugs  sex or violence  its is hardcore  in the classic use of the word it is called oz as that is the nickname given to the oswald maximum security state penitentary  it focuses mainly on emerald city  an experimental section of the prison where all the cells have glass fronts and face inwards  so privacy is not high on the agenda  em city is home to many  aryans  muslims  gangstas  latinos  christians  italians  irish and more    so scuffles  death stares  dodgy dealings and shady agreements are never far away i would say the main appeal of the show is due to the fact that it goes where other shows wo

In [17]:
# If you need to install NLTK
!pip install nltk


In [18]:
from nltk.corpus import stopwords

# Get the list of English stopwords to confirm download success
english_stopwords = stopwords.words("english")

# Print a few stopwords to confirm it works
print("Sample stopwords:", english_stopwords[:10])


Sample stopwords: ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [44]:
import pandas as pd
import nltk
from nltk.corpus import stopwords

# Download the stopwords if not already downloaded
nltk.download("stopwords")

# Get the list of English stopwords
english_stopwords = set(stopwords.words("english"))

# Read the CSV file into a DataFrame
data = pd.read_csv("/IMDB-Dataset.csv")

# Check if the 'review' column exists and create a new column without stopwords
if 'review' in data.columns:
    def remove_stopwords(text):
        # Tokenize the text into words
        words = text.lower().split()
        # Filter out stopwords
        filtered_words = [word for word in words if word not in english_stopwords]
        # Join the filtered words into a single string
        return " ".join(filtered_words)

    # Apply the function to remove stopwords from the 'review' column
    data['review_no_stopwords'] = data['review'].apply(remove_stopwords)

    # Show the first few rows of the DataFrame to confirm the result
    print(data[['review', 'review_no_stopwords']].head())
else:
    print("Error: The 'review' column is not found in the CSV.")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                              review  \
0  One of the other reviewers has mentioned that ...   
1  A wonderful little production. <br /><br />The...   
2  I thought this was a wonderful way to spend ti...   
3  Basically there's a family where a little boy ...   
4  Petter Mattei's "Love in the Time of Money" is...   

                                 review_no_stopwords  
0  one reviewers mentioned watching 1 oz episode ...  
1  wonderful little production. <br /><br />the f...  
2  thought wonderful way spend time hot summer we...  
3  basically there's family little boy (jake) thi...  
4  petter mattei's "love time money" visually stu...  


In [45]:
def stem_txt(text):
    ss = SnowballStemmer('english')
    return " ".join([ss.stem(w) for w in text])

data.review = data.review.apply(stem_txt)
data.review[0]

"o n e   o f   t h e   o t h e r   r e v i e w e r s   h a s   m e n t i o n e d   t h a t   a f t e r   w a t c h i n g   j u s t   1   o z   e p i s o d e   y o u ' l l   b e   h o o k e d .   t h e y   a r e   r i g h t ,   a s   t h i s   i s   e x a c t l y   w h a t   h a p p e n e d   w i t h   m e . < b r   / > < b r   / > t h e   f i r s t   t h i n g   t h a t   s t r u c k   m e   a b o u t   o z   w a s   i t s   b r u t a l i t y   a n d   u n f l i n c h i n g   s c e n e s   o f   v i o l e n c e ,   w h i c h   s e t   i n   r i g h t   f r o m   t h e   w o r d   g o .   t r u s t   m e ,   t h i s   i s   n o t   a   s h o w   f o r   t h e   f a i n t   h e a r t e d   o r   t i m i d .   t h i s   s h o w   p u l l s   n o   p u n c h e s   w i t h   r e g a r d s   t o   d r u g s ,   s e x   o r   v i o l e n c e .   i t s   i s   h a r d c o r e ,   i n   t h e   c l a s s i c   u s e   o f   t h e   w o r d . < b r   / > < b r   / > i t   i s   c a l l e d   o z

In [46]:
data.head()

,review,sentiment,review_no_stopwords
0,o n e o f t h e o t h e r r e v i e w ...,positive,one reviewers mentioned watching 1 oz episode ...
1,a w o n d e r f u l l i t t l e p r o d ...,positive,wonderful little production. <br /><br />the f...
2,i t h o u g h t t h i s w a s a w o ...,positive,thought wonderful way spend time hot summer we...
3,b a s i c a l l y t h e r e ' s a f a m ...,negative,basically there's family little boy (jake) thi...
4,"p e t t e r m a t t e i ' s "" l o v e i ...",positive,"petter mattei's ""love time money"" visually stu..."


In [50]:
# Check for missing or null values in the 'review' column
missing_reviews = data['review'].isnull().sum()
print("Number of missing or null reviews:", missing_reviews)

# Remove rows with missing or null reviews
data = data.dropna(subset=['review'])

# Check for entirely empty reviews (only whitespace)
empty_reviews = data['review'].apply(lambda x: x.strip() == "").sum()
print("Number of empty reviews:", empty_reviews)

# Remove empty reviews
if empty_reviews > 0:
    data = data[data['review'].apply(lambda x: x.strip() != "")]


Number of missing or null reviews: 0
Number of empty reviews: 0


In [51]:
import re

# Function to clean text by removing special characters and converting to lowercase
def preprocess_text(text):
    # Remove anything that's not a letter or space
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower().strip()
    return text

# Apply the preprocessing function to the 'review' column
data['cleaned_review'] = data['review'].apply(preprocess_text)

# Display some cleaned reviews to ensure they contain valid content
print("Sample cleaned reviews:")
print(data['cleaned_review'].head())


Sample cleaned reviews:
0    o n e   o f   t h e   o t h e r   r e v i e w ...
1    a   w o n d e r f u l   l i t t l e   p r o d ...
2    i   t h o u g h t   t h i s   w a s   a   w o ...
3    b a s i c a l l y   t h e r e  s   a   f a m i...
4    p e t t e r   m a t t e i  s    l o v e   i n ...
Name: cleaned_review, dtype: object


In [55]:
# Install scikit-learn with the correct package name
!pip install scikit-learn


In [57]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

# Download NLTK stopwords if necessary
import nltk
nltk.download("stopwords")

# Load the CSV file
data = pd.read_csv("/IMDB-Dataset.csv")

# Ensure 'review' column is not empty or missing
data = data.dropna(subset=['review'])  # Remove rows with missing reviews
data = data[data['review'].apply(lambda x: len(x.strip()) > 0)]  # Remove empty reviews

# Function to clean text by removing special characters and stop words
def clean_text(text):
    # Keep only letters and spaces, then convert to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text).lower().strip()
    # Remove stop words
    stop_words = set(stopwords.words("english"))
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text

# Apply the cleaning function to the 'review' column
data['cleaned_review'] = data['review'].apply(clean_text)

# Check for valid content in cleaned reviews
if data['cleaned_review'].apply(lambda x: len(x.strip()) > 0).all():
    # Apply CountVectorizer to the cleaned text
    cv = CountVectorizer(max_features=1000)
    X = cv.fit_transform(data['cleaned_review']).toarray()
    y = np.array(data['sentiment'].values)  # Assuming 'sentiment' is the target variable

    print("X.shape =", X.shape)  # Display the shape of the feature matrix
    print("y.shape =", y.shape)  # Display the shape of the target array
else:
    print("Error: No valid content found in the 'cleaned_review' column.")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


X.shape = (50000, 1000)
y.shape = (50000,)


In [58]:
print(X)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [59]:
trainx,testx,trainy,testy = train_test_split(X,y,test_size=0.2,random_state=9)
print("Train shapes : X = {}, y = {}".format(trainx.shape,trainy.shape))
print("Test shapes : X = {}, y = {}".format(testx.shape,testy.shape))

Train shapes : X = (40000, 1000), y = (40000,)
Test shapes : X = (10000, 1000), y = (10000,)


In [60]:
gnb,mnb,bnb = GaussianNB(),MultinomialNB(alpha=1.0,fit_prior=True),BernoulliNB(alpha=1.0,fit_prior=True)
gnb.fit(trainx,trainy)
mnb.fit(trainx,trainy)
bnb.fit(trainx,trainy)

BernoulliNB()

In [76]:
# If necessary, install required packages
!pip install numpy pandas scikit-learn


In [77]:
import pickle

# Assuming you have trained models gnb, mnb, bnb
# Save each model to a separate file
pickle.dump(gnb, open("gaussian_nb.pkl", "wb"))  # Save GaussianNB
pickle.dump(mnb, open("multinomial_nb.pkl", "wb"))  # Save MultinomialNB
pickle.dump(bnb, open("bernoulli_nb.pkl", "wb"))  # Save BernoulliNB


In [87]:
# Load the GaussianNB model
loaded_gnb = pickle.load(open("gaussian_nb.pkl", "rb"))

# Load the MultinomialNB model
loaded_mnb = pickle.load(open("multinomial_nb.pkl", "rb"))

# Load the BernoulliNB model
loaded_bnb = pickle.load(open("bernoulli_nb.pkl", "rb"))


In [88]:
ypg = gnb.predict(testx)
ypm = mnb.predict(testx)
ypb = bnb.predict(testx)

print("Gaussian = ",accuracy_score(testy,ypg))
print("Multinomial = ",accuracy_score(testy,ypm))
print("Bernoulli = ",accuracy_score(testy,ypb))

Gaussian =  0.8112
Multinomial =  0.8314
Bernoulli =  0.8246


In [89]:
pickle.dump(bnb,open('model1.pkl','wb'))

In [90]:
# If you need to install NLTK
!pip install nltk


In [91]:
import nltk

# Download the punkt resource
nltk.download("punkt")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [92]:
rev =  """Amazing season Hopefully Season one is as good as this one"""

f1 = clean(rev)
f2 = is_special(f1)
f3 = to_lower(f2)
f4 = rem_stopwords(f3)
f5 = stem_txt(f4)

bow,words = [],word_tokenize(f5)
for word in words:
    bow.append(words.count(word))
#np.array(bow).reshape(1,3000)
#bow.shape
word_dict = cv.vocabulary_
pickle.dump(word_dict,open('bow.pkl','wb'))

In [98]:
inp = ["""Amazing season Hopefully Season one is as good as this one"""]
for i in word_dict:
    inp.append(f5.count(i[0]))
y_pred = bnb.predict(np.array(inp).reshape(1,1000))

ValueError: cannot reshape array of size 1001 into shape (1,1000)

In [102]:
# If necessary, install required packages
!pip install sklearn pandas


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [107]:
import re
import pickle
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk


# Example training data
data = pd.DataFrame({"review": ["Great movie!", "Not so good", "Loved it!", "It was terrible"]})

# Create and fit a CountVectorizer
cv = CountVectorizer(max_features=1000)  # Limiting to 1000 features
cv.fit(data["review"])  # Train the vectorizer on example data

# Create a training set for GaussianNB
train_data = cv.transform(data["review"]).toarray()  # Transform reviews into feature vectors
train_labels = [1, 0, 1, 0]  # Example binary labels for sentiment (positive/negative)

# Train the GaussianNB model
gnb = GaussianNB()
gnb.fit(train_data, train_labels)

# Save the GaussianNB model to a file
pickle.dump(gnb, open("gaussian_nb.pkl", "wb"))  # Save the model


In [111]:
# Load the GaussianNB model
gnb = pickle.load(open("gaussian_nb.pkl", "rb"))  # Load the saved GaussianNB model

# Example review to predict
new_review = "I really enjoyed the movie, it was fantastic!"

# Clean and preprocess the review text
stop_words = set(stopwords.words("english"))

# Function to clean the text
def clean_text(text):
    text = re.sub(r"[^a-zA-Z\s]", "", text).lower().strip()  # Remove non-letters and lowercase
    words = [word for word in text.split() if word not in stop_words]  # Remove stop words
    return " ".join(words)

# Clean and transform the text into a feature vector
cleaned_review = clean_text(new_review)
feature_vector = cv.transform([cleaned_review]).toarray()  # Transform to feature vector

# Make a prediction with the GaussianNB model
prediction = gnb.predict(feature_vector)  # Predict with GaussianNB

# Output the prediction
print("Prediction for the given review:", prediction)


Prediction for the given review: [1]


In [105]:
# List files in the current directory to confirm 'vectorizer.pkl' exists
!ls -l


total 140
-rw-r--r-- 1 root root 32664 Apr 30 01:40 bernoulli_nb.pkl
-rw-r--r-- 1 root root 27807 Apr 30 01:43 bow.pkl
drwx------ 5 root root  4096 Apr 30 01:02 drive
-rw-r--r-- 1 root root   796 Apr 30 01:52 gaussian_nb.pkl
-rw-r--r-- 1 root root 32664 Apr 30 01:43 model1.pkl
-rw-r--r-- 1 root root 32646 Apr 30 01:40 multinomial_nb.pkl
drwxr-xr-x 1 root root  4096 Apr 26 13:22 sample_data
-rw-r--r-- 1 root root   796 Apr 30 01:51 vectorizer.pkl
